# Import Libraries



In [ ]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
import numbers

#Get connected to Google Drive

In [ ]:
# This is to connect to your google drive so you can load the dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Before running the following command, please create a directory "DataMining/2022F/A2/Data" under "My Drive" on your Google Drive account and put the date files (such as claim.csv) there. 
# You are requred to use this directory for the marking purpose so that the TA can read the data from the same directory for all the submitted programs. 
%cd /content/drive/MyDrive/DataMining/2022F/A2/Data

#Load Data

In [ ]:
data = pd.read_csv("claim.csv")

In [ ]:
data

In [ ]:
data.describe()

# Code Implementation

##Calculate entropy

Compelete the following function, which takes a dataset and the class attribute of the dataset and returns the enthropy of the dataset.

In [ ]:
# The input should be a dataset stored in a Pandas' DataFrame (such as data in the above code) and class attribute column (such as data.Outcome)
def calculate_entropy(dataset, class_attribute):
    entropy = 0
    values = class_attribute.unique()
    for value in values:
        fraction = class_attribute.value_counts()[value]/len(class_attribute)
        entropy += -fraction*np.log2(fraction)
    return entropy

In [ ]:
# An example of calling the calculate_entropy function to compute the entropy of the dataset stored in variable data with respect to the class attribute Outcome.
calculate_entropy(data, data.Outcome)

## Calculate information gain

Compute the information gain of an input attribute. The function takes a dataset, its class attribute, and an input attribute and returns the information again of the input attribute. 

This function should call the calculate_entropy function defined above.

Note that there are two types of input attributes (symbolic and numeric). They should be treated differently.

In [ ]:
def calculate_gain(dataset, class_attribute, input_attribute):

  target_variables = class_attribute.unique()

  if isinstance(input_attribute[0],numbers.Number):
    x =[]
    for i in range (class_attribute.shape[0]-1):
      if class_attribute[i] != class_attribute[i+1]:
        x.append((input_attribute[i]+input_attribute[i+1])/2)
    print("Splitting points are:\n",x)

    I=[]
    for point in x:
      a = input_attribute.copy()
      for i in range (class_attribute.shape[0]):
        if input_attribute[i] <= point:
          a[i] = "up"
        else:
          a[i] = "down"
      variables = a.unique()
      entropy2 = 0
      for variable in variables:
          entropy = 0
          for target_variable in target_variables:
              num = len(a[a==variable][class_attribute ==target_variable])
              den = len(a[a==variable])
              fraction = num/(den+eps)
              entropy += -fraction*log(fraction+eps)
          fraction2 = den/len(dataset)
          entropy2 += -fraction2*entropy
          IG = calculate_entropy(dataset, class_attribute) - abs(entropy2)
      I.append(IG)
    print("Gain is: ")
    return I[np.argmax(I)]
  
  else:
 
    variables = input_attribute.unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(input_attribute[input_attribute==variable][class_attribute ==target_variable])
            den = len(input_attribute[input_attribute==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(dataset)
        entropy2 += -fraction2*entropy
    IG = calculate_entropy(dataset, class_attribute) - abs(entropy2)
    return IG

In [ ]:
# An example of calling the calculate_gain function to compute the information gain of the Age attribute
calculate_gain(data, data.Outcome, data.ClaimAmount)

## Attribute Selection

Now, write a function that reads the data and 
uses information gain to choose the attribute 
for the decision tree root. The function returns the name (such as Age) of the selected attribute if the attribute is a symbolic attribute, or the name of the attribute plus its binary split point if the attribute is a numeric attribute (e.g., ClaimAmount with 4000)

In [ ]:
def select_attribute(dataset, class_attribute):
    Entropy_att = []
    IG = []
    for key in dataset.keys()[:-1]:
        IG.append(calculate_gain(dataset, class_attribute, dataset[key]))
    return dataset.keys()[:-1][np.argmax(IG)]

# Test program
Calls the select_attribute function with the training data (stored in data) and class attribute to find the best input attribute for the root node of a decision tree. Your program may be tested on other datasets.

In [ ]:
select_attribute(data, data.Outcome)